In [2]:
using Pkg
include("./P2H_CapacityExpansion.jl")
cd("/cluster/home/danare/git")
Pkg.activate(".")
using .P2H_CapacityExpansion
using DataFrames
using Parameters
using Flux
using Surrogates
using ScikitLearn
using LinearAlgebra, Random, Statistics
using JuMP
using XLSX
using MathOptAI
using PlotlyJS
using Ipopt
using Clustering
using CSV
using Dates
using StatsBase, MultivariateStats

  Activating project at `~/git`


# 1. Read in the data

In [3]:
file = "/cluster/home/danare/git/P2H_CapacityExpansion/results/500_scenarios_V3.txt"

"/cluster/home/danare/git/P2H_CapacityExpansion/results/500_scenarios_V3.txt"

In [4]:
df = P2H_CapacityExpansion.read_txt_file(file);
df = select(df, Not(:ENS))
size(df)

(1500, 15)

# 2. Split the Data into Training and Test

In [5]:
X_train, y_train, X_test, y_test = P2H_CapacityExpansion.partitionTrainTest(df, [:Cost,:Generation, :Emission], 0.7)

### scale the data ###
X_train_scaled, μX, σX  = P2H_CapacityExpansion.scaling(X_train)
X_test_scaled = (X_test .- μX) ./ σX
y_train_scaled, μy, σy  = P2H_CapacityExpansion.scaling(y_train)
    
# remove np.nan #
for i in eachindex(X_test_scaled)
    if isnan(X_test_scaled[i])
        X_test_scaled[i] = 0.0
    end
end

# 3. Train the Surrogate Model

In [6]:
### train ML model and compute R2 ### 
sg = P2H_CapacityExpansion.neural_network_model_flux(X_train_scaled, y_train_scaled, X_test_scaled, y_test, σy, μy; hidden_layer=500, epochs=100)

┌ Warning: Layer with Float32 parameters got Float64 input.
│   The input will be converted, but any earlier layers may be very slow.
│   layer = Dense(12 => 500, relu)
│   summary(x) = 12×32 Matrix{Float64}
└ @ Flux /cluster/home/danare/.julia/packages/Flux/hiqg1/src/layers/stateless.jl:60


Epoch = 1 : Training loss = 0.2235748562321015, R2 score : 0.9687247801882584
Epoch = 2 : Training loss = 0.162899993193753, R2 score : 0.9848429962732638
Epoch = 3 : Training loss = 0.13074388488791194, R2 score : 0.9923484146040519
Epoch = 4 : Training loss = 0.11316863062894518, R2 score : 0.9917783925790882
Epoch = 5 : Training loss = 0.09759323065157448, R2 score : 0.9948492982929152
Epoch = 6 : Training loss = 0.0939522445638875, R2 score : 0.9945241232773744
Epoch = 7 : Training loss = 0.0893676149295337, R2 score : 0.9946388730421675
Epoch = 8 : Training loss = 0.08404186551152454, R2 score : 0.9938131335807555
Epoch = 9 : Training loss = 0.080763129915028, R2 score : 0.9955717679892564
Epoch = 10 : Training loss = 0.085121162531885, R2 score : 0.9954227607742229
Epoch = 11 : Training loss = 0.07375674937440521, R2 score : 0.9965179476705324
Epoch = 12 : Training loss = 0.07710580484262741, R2 score : 0.9960374897428491
Epoch = 13 : Training loss = 0.07502724095637286, R2 score

Main.P2H_CapacityExpansion.Surrogate(Chain(Dense(12 => 500, relu), Dense(500 => 3)), Float32[-0.09269529 0.023661204 0.5819961; -0.093335144 0.054316446 0.59909123; … ; -0.103215314 0.08897336 0.62995565; -0.21935277 -0.19978185 0.8653148])

# 4. Read the Input Data

In [28]:
# load input
config = P2H_CapacityExpansion.read_yaml_file();
data = P2H_CapacityExpansion.load_cep_data(config=config);
ts_data = P2H_CapacityExpansion.load_timeseries_data_full(config=config);

┌ Info: Each time-series is averaged in 5-hourly steps
└ @ Main.P2H_CapacityExpansion /cluster/home/danare/git/P2H_CapacityExpansion/utils/load_data.jl:76


## 5. Instantiate the Model

In [30]:
cep = P2H_CapacityExpansion.run_opt(ts_data=ts_data, data=data, config=config, surrogate=true, solver=Ipopt.Optimizer)

@unpack 𝓖, 𝓨, 𝓣, 𝓡, 𝓢, 𝓛, 𝓒 = P2H_CapacityExpansion.get_sets(cep=cep)

##################### cost optimization #####################

P2H_CapacityExpansion.setup_opt_costs_fix!(cep, config, data.data,vcat(cep.sets["non_dispatch"], cep.sets["dispatch"], 𝓢, String[s for s in cep.sets["discharging"]], cep.sets["conversion"]))

#JuMP.fix.(cep.model[:COST]["var", :, :], 0; force=true);
@variable(cep.model, COST_VAR[y ∈ 𝓨] ≥ 0);

┌ Info: Reading the data ...
└ @ Main.P2H_CapacityExpansion /cluster/home/danare/git/P2H_CapacityExpansion/src/opt.jl:19
┌ Info: Investment mode is on.
└ @ Main.P2H_CapacityExpansion /cluster/home/danare/git/P2H_CapacityExpansion/src/opt.jl:88


# 6. Embedd the Surogate into the Optimization Model

In [36]:
techs = [f for f ∈ cep.sets["invest_tech"] if f != "ENS"]

for y ∈ 𝓨, r ∈ 𝓡
    x_vec = [cep.model[:TotalCapacityAnnual][r, g, y] for g ∈ techs]
    x_scaled = (x_vec ) #.- μX) ./ σX
    prediction, formulation = MathOptAI.add_predictor(cep.model, sg.model, x_scaled)
    y_rescaled = prediction #.* σy .+ μy

    ##### Cost approximation ###
    @constraint(cep.model, COST_VAR[y] .>= y_rescaled[1])

    ##### Generation  ###
    #TODO H2 and electricity combined as sum
    @constraint(cep.model, y_rescaled[2] .>= data.data["demand"][r,y,"electricity"] + data.data["demand"][r,y,"H2"])

    ##### Emission reduction  ###
    @constraint(cep.model, sum(data.data["budget"][r,y] for r ∈ 𝓡).>= y_rescaled[3] )
end

In [37]:
opex_discounted = sum(
    1 / ((1 + config["r"])^(y - 𝓨[1] - 10)) * (
        sum(cep.model[:COST]["fix", y, g] for g ∈ 𝓖) +
        COST_VAR[y]
    ) for y ∈ 𝓨
)

@objective(cep.model, Min, sum(
    1 / ((1 + config["r"])^(y - 𝓨[1] - 10)) *
    sum(cep.model[:COST]["cap", y, g] for g ∈ cep.sets["invest_all"])
    for y ∈ 𝓨 
) + opex_discounted)

1.4802442849183444 COST[cap,2030,P_Wind_Offshore_Transitional] + 1.4802442849183444 COST[cap,2030,S_Battery_Li-Ion] + 1.4802442849183444 COST[cap,2030,P_Wind_Onshore_Avg] + 1.4802442849183444 COST[cap,2030,X_Electrolysis] + 1.4802442849183444 COST[cap,2030,trans_H2] + 1.4802442849183444 COST[cap,2030,P_PV_Utility_Avg] + 1.4802442849183444 COST[cap,2030,P_Gas_CCGT] + 1.4802442849183444 COST[cap,2030,D_Gas_H2_out] + 1.4802442849183444 COST[cap,2030,trans_elec] + 1.4802442849183444 COST[cap,2030,D_Battery_Li-Ion_out] + 1.4802442849183444 COST[cap,2030,ENS] + 1.4802442849183444 COST[cap,2030,P_Biomass] + 1.4802442849183444 COST[cap,2030,S_Gas_H2] + 1.4802442849183444 COST[cap,2030,P_H2_OCGT] + 1.4802442849183444 COST[cap,2030,X_ATR_CCS] + COST[cap,2040,P_Wind_Offshore_Transitional] + COST[cap,2040,S_Battery_Li-Ion] + COST[cap,2040,P_Wind_Onshore_Avg] + COST[cap,2040,X_Electrolysis] + COST[cap,2040,trans_H2] + COST[cap,2040,P_PV_Utility_Avg] + COST[cap,2040,P_Gas_CCGT] + COST[cap,2040,D_Gas

In [ ]:
result = P2H_CapacityExpansion.optimize_and_output(cep=cep, config=config, data=data, ts_data=ts_data, name="scenario_v3", short_sol=false)

In [50]:
prediction

1-element Vector{VariableRef}:
 moai_Affine[1]

In [36]:
println("Total variables: ", JuMP.num_variables(cep.model))
println("Total constraints: ", JuMP.num_constraints(cep.model))

Total variables: 6369


UndefKeywordError: UndefKeywordError: keyword argument `count_variable_in_set_constraints` not assigned

In [53]:
typeof(z)

VariableRef (alias for GenericVariableRef{Float64})